In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import json
import sklearn

In [2]:
# embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
# embeddings = embed([
#     "The quick brown fox jumps over the lazy dog.",
#     "I am a sentence for which I would like to get its embedding"])

# print(embeddings)

In [3]:
df = pd.read_csv("steam_sample.csv", 
                 header=None,
                 names=["GameID", "Review", "Recommend", "FoundHelpful"])
df.loc[df['Recommend'] == 1, 'Sentiment'] = 1.0
df.loc[df['Recommend'] == -1, 'Sentiment'] = 0.0
df.drop(columns=["GameID", "Recommend", "FoundHelpful"], inplace=True)
# show a few examples
df.head()

,Review,Sentiment
0,Ruined my life.,1.0
1,This will be more of a ''my experience with th...,1.0
2,This game saved my virginity.,1.0
3,• Do you like original games? • Do you like ga...,1.0
4,"Easy to learn, hard to master.",1.0


In [4]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
from autocorrect import Speller

def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    [s.extract() for s in soup(['iframe', 'script'])]
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
    return stripped_text

def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def expand_contractions(text):
    return contractions.fix(text)

def remove_special_characters(text, remove_digits=False):
    pattern = r'[^a-zA-Z0-9\s]' if not remove_digits else r'[^a-zA-Z\s]'
    text = re.sub(pattern, '', text)
    return text

def correct_spelling(text):
    spell = Speller(lang='en')
    return spell(text)
            
def pre_process_document(document):
    # strip HTML
    document = strip_html_tags(document)
    # lower case
    document = document.lower()
    # remove extra newlines (often might be present in really noisy text)
    document = document.translate(document.maketrans("\n\t\r", "   "))
    # remove accented characters
    document = remove_accented_chars(document)
    # expand contractions    
    document = expand_contractions(document)  
    # remove special characters and\or digits    
    # insert spaces between special characters to isolate them    
    special_char_pattern = re.compile(r'([{.(-)!}])')
    document = special_char_pattern.sub(" \\1 ", document)
    document = remove_special_characters(document, remove_digits=True)  
    # remove extra whitespace
    document = re.sub(' +', ' ', document)
    document = document.strip()
    # correct spelling
    # document = correct_spelling(document)
    
    return document


pre_process_corpus = np.vectorize(pre_process_document)

In [5]:
# negative reviews examples
df[df["Sentiment"]==0.0].head()

,Review,Sentiment
3330,"You are 14 years old, first time trying to pla...",0.0
3725,Game full of cheaters nice valve.,0.0
4357,"Eh, its alright",0.0
4366,"Eh, its alright",0.0
4523,full of russians,0.0


In [6]:
# there are null reviews
df[df["Review"].isnull()].head()
# df[df["Sentiment"].isnull()]
# df[df["Review"].isna()]
# df[df["Sentiment"].isnull()]

,Review,Sentiment
505,NaN,1.0
539,NaN,1.0
540,NaN,1.0
588,NaN,1.0
675,NaN,1.0


In [7]:
# replace null reviews with null strings
df.loc[df["Review"].isnull(), 'Review'] = ""

In [8]:
# preprocess by replacing html and accented/special characters
df.loc[:, 'Review'] = pre_process_corpus(df["Review"])

/Users/deanwang/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/deanwang/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/deanwang/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'//'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/deanwang/miniconda3/lib/python3.7/site-packages/bs4/__init__.py:314: UserWarning: "b'/'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/Users/deanwang/miniconda3/lib

In [9]:
# split into test and train
msk = np.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]
print(len(train_df))
print(len(test_df))

79886
20114


In [10]:
# Training input on the whole training set with no limit on training epochs.
train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["Sentiment"], num_epochs=None, shuffle=True)

In [11]:
# Prediction on the whole training set.
predict_train_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    train_df, train_df["Sentiment"], shuffle=False)
# Prediction on the test set.
predict_test_input_fn = tf.compat.v1.estimator.inputs.pandas_input_fn(
    test_df, test_df["Sentiment"], shuffle=False)

In [12]:
embedded_text_feature_column = hub.text_embedding_column(
    key="Review", 
    module_spec="https://tfhub.dev/google/nnlm-en-dim128/1")

In [13]:
tf.keras.backend.set_floatx('float32')
estimator = tf.estimator.DNNClassifier(
    hidden_units=[500, 100],
    feature_columns=[embedded_text_feature_column],
    n_classes=2,
    optimizer=tf.keras.optimizers.Adagrad(lr=0.003))

INFO:tensorflow:Using default config.


INFO:tensorflow:Using default config.


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [14]:
# Training for 5,000 steps means 640,000 training examples with the default
# batch size. This is roughly equivalent to 25 epochs since the training dataset
# contains 25,000 examples.
estimator.train(input_fn=train_input_fn, steps=5000)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.


Instructions for updating:
To construct input pipelines, use the `tf.data` module.


...

INFO:tensorflow:loss = 0.088730626, step = 4900 (0.822 sec)


INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu/model.ckpt.


INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu/model.ckpt.


INFO:tensorflow:Loss for final step: 0.057810992.


INFO:tensorflow:Loss for final step: 0.057810992.


In [15]:
train_eval_result = estimator.evaluate(input_fn=predict_train_input_fn)
test_eval_result = estimator.evaluate(input_fn=predict_test_input_fn)

print("Training set accuracy: {accuracy}".format(**train_eval_result))
print("Test set accuracy: {accuracy}".format(**test_eval_result))

INFO:tensorflow:Calling model_fn.


INFO:tensorflow:Calling model_fn.



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8627821, accuracy_baseline = 0.8627821, auc = 0.80217785, auc_precision_recall = 0.9563355, average_loss = 0.37136826, global_step = 5000, label/mean = 0.8627821, loss = 0.36951572, precision = 0.86281824, prediction/mean = 0.94010043, recall = 0.99994236


INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.8627821, accuracy_baseline = 0.8627821, auc = 0.80217785, auc_precision_recall = 0.9563355, average_loss = 0.37136826, global_step = 5000, label/mean = 0.8627821, loss = 0.36951572, precision = 0.86281824, prediction/mean = 0.94010043, recall = 0.99994236


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu/model.ckpt-5000


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /var/folders/j9/9tb4b_ds1r14clxss85c8xq80000gn/T/tmpe71bvkbu/model.ckpt-5000


Training set accuracy: 0.8647948503494263
Test set accuracy: 0.8627821207046509
